# Supervised Learning Technique 

This notebook show how Supervised Machine Learning techniques can be used to predict the Loan defaulters

Algorithms used:<br>
Logistic regression <br>
Random Forest <br>
Extreme Gradient Boost <br>

Also I have used SMOTE and Random OverSampler to balance the class.

### Load the data

In [1]:
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

import sys
import warnings
warnings.filterwarnings("ignore")

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

In [2]:
application_train = pd.read_csv('cleandata.csv')

## Modeling

### Partitioning

In [3]:
#Determine the number of fraud and valid transactions in the dataset

Fraud = application_train[application_train['TARGET']==1]

Valid = application_train[application_train['TARGET']==0]

outlier_fraction = len(Fraud)/float(len(Valid))

print('outlier_fraction for the whole dataset:')
print(outlier_fraction)

print("Fraud Cases : {}".format(len(Fraud)))

print("Valid Cases : {}".format(len(Valid)))

outlier_fraction for the whole dataset:
0.08781828601345662
Fraud Cases : 24825
Valid Cases : 282686


In [4]:
from sklearn.model_selection import train_test_split

#Create independent and Dependent Features
columns = application_train.columns.tolist()
# Filter the columns to remove data we do not want 
columns = [c for c in columns if c not in ["TARGET"]]
# Store the variable we are predicting 
target = "TARGET"
# Define a random state 
state = np.random.RandomState(42)
X = application_train[columns]
y = application_train[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(246008, 210) (246008,)
(61503, 210) (61503,)


### More Feature Selection

using RandomForestClassifier

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [7]:
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100, random_state=state))
sel.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier(random_state=RandomState(MT19937) at 0x2211291CD08))

In [8]:
sel.get_support()

array([False, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False, False, False, False,  True,
        True, False, False, False, False, False,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True, False,  True,  True, False,  True,  True,
        True, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
        True,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False,

In [9]:
selected_feat= X_train.columns[(sel.get_support())].tolist()
len(selected_feat)

42

In [10]:
print(selected_feat)

['FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAREA_AVG', 'TOTALAREA_MODE', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'NAME_FAMILY_STATUS_Married', 'OCCUPATION_TYPE_Laborers', 'CREDIT_INCOME_PERCENT', 'ANNUITY_INCOME_PERCENT', 'CREDIT_TERM', 'DAYS_EMPLOYED_PERCENT']


In [11]:
app_X_train = X_train.copy() #Taking a copy before dropping 
app_X_test = X_test.copy()

In [12]:
X_train = X_train[selected_feat]
X_test = X_test[selected_feat]

In [13]:
print (X_train.shape, X_test.shape)

(246008, 42) (61503, 42)


------------------------------

## Logistic Regression

In [15]:

from sklearn.linear_model import LogisticRegression

logistic_regressor = LogisticRegression(C = 2)

In [16]:
logistic_regressor.fit(X_train,y_train)


LogisticRegression(C=2)

In [17]:
y_pred = logistic_regressor.predict(X_test)

In [18]:
from sklearn.metrics import classification_report,accuracy_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix

n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Logistic Regression errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Logistic Regression errors: 4946
Accuracy Score :
0.9195811586426679
Confusion matrix :
[[56557     3]
 [ 4943     0]]
Classification Report :
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56560
           1       0.00      0.00      0.00      4943

    accuracy                           0.92     61503
   macro avg       0.46      0.50      0.48     61503
weighted avg       0.85      0.92      0.88     61503

ROC AUC score is:  0.4999734794908062


## Random Forest - Bagging ensemble of Decision trees

In [19]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators = 100, random_state = state, verbose = 1, n_jobs = -1)

In [20]:
random_forest.fit(X_train,y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   29.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.1min finished


RandomForestClassifier(n_jobs=-1,
                       random_state=RandomState(MT19937) at 0x2211291CD08,
                       verbose=1)

In [21]:
y_pred = random_forest.predict(X_test) 

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.0s finished


In [22]:
n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Random Forest errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Random Forest errors: 4937
Accuracy Score :
0.9197274929678227
Confusion matrix :
[[56543    17]
 [ 4920    23]]
Classification Report :
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56560
           1       0.57      0.00      0.01      4943

    accuracy                           0.92     61503
   macro avg       0.75      0.50      0.48     61503
weighted avg       0.89      0.92      0.88     61503

ROC AUC score is:  0.5021762394694138


## Extreme Gradient Boost Model

In [23]:
from xgboost import XGBClassifier

In [24]:
xgb_classifier = XGBClassifier(n_estimators=100,max_depth=5)

In [25]:
xgb_classifier.fit(X_train,y_train)

XGBClassifier(max_depth=5)

In [26]:
y_pred = xgb_classifier.predict(X_test)

In [27]:
n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Extreme Gradient Boost errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Extreme Gradient Boost errors: 4927
Accuracy Score :
0.9198900866624392
Confusion matrix :
[[56520    40]
 [ 4887    56]]
Classification Report :
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56560
           1       0.58      0.01      0.02      4943

    accuracy                           0.92     61503
   macro avg       0.75      0.51      0.49     61503
weighted avg       0.89      0.92      0.88     61503

ROC AUC score is:  0.5053109693790685


# Balancing class - using SMOTETomek

I have used SMOTETomek which is a method of imblearn. SMOTETomek is a hybrid method which uses an under sampling method (Tomek) in with an over sampling method (SMOTE).

In [28]:
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss

In [29]:
# Implementing Oversampling for Handling Imbalanced 
smk = SMOTETomek(random_state=42)
X_res,y_res=smk.fit_sample(X_train,y_train)

In [30]:
X_res.shape,y_res.shape

((447264, 42), (447264,))

In [31]:
from collections import Counter
print('Original dataset shape {}'.format(Counter(y_train)))
print('Resampled dataset shape {}'.format(Counter(y_res)))

Original dataset shape Counter({0: 226126, 1: 19882})
Resampled dataset shape Counter({0: 223632, 1: 223632})


## Logistic regression

In [32]:
logistic_regressor.fit(X_res,y_res)

LogisticRegression(C=2)

In [33]:
y_pred = logistic_regressor.predict(X_test)

In [34]:
n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Logistic Regression errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Logistic Regression errors: 23015
Accuracy Score :
0.6257906118400729
Confusion matrix :
[[35929 20631]
 [ 2384  2559]]
Classification Report :
              precision    recall  f1-score   support

           0       0.94      0.64      0.76     56560
           1       0.11      0.52      0.18      4943

    accuracy                           0.63     61503
   macro avg       0.52      0.58      0.47     61503
weighted avg       0.87      0.63      0.71     61503

ROC AUC score is:  0.576469358537397


## Random forest

In [35]:
random_forest.fit(X_res,y_res)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   57.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.1min finished


RandomForestClassifier(n_jobs=-1,
                       random_state=RandomState(MT19937) at 0x2211291CD08,
                       verbose=1)

In [36]:
y_pred = random_forest.predict(X_test)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.1s finished


In [37]:
n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Random Forest errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Random Forest errors: 5226
Accuracy Score :
0.9150285351934052
Confusion matrix :
[[56165   395]
 [ 4831   112]]
Classification Report :
              precision    recall  f1-score   support

           0       0.92      0.99      0.96     56560
           1       0.22      0.02      0.04      4943

    accuracy                           0.92     61503
   macro avg       0.57      0.51      0.50     61503
weighted avg       0.86      0.92      0.88     61503

ROC AUC score is:  0.5078372852927905


## XGBoost

In [38]:
X_res = pd.DataFrame(data=X_res, columns=selected_feat)

In [39]:
xgb_classifier.fit(X_res,y_res)

XGBClassifier(max_depth=5)

In [40]:
y_pred = xgb_classifier.predict(X_test)

In [41]:
n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Extreme Gradient Boost errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Extreme Gradient Boost errors: 5088
Accuracy Score :
0.9172723281791132
Confusion matrix :
[[56303   257]
 [ 4831   112]]
Classification Report :
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56560
           1       0.30      0.02      0.04      4943

    accuracy                           0.92     61503
   macro avg       0.61      0.51      0.50     61503
weighted avg       0.87      0.92      0.88     61503

ROC AUC score is:  0.5090572287157041


# Balancing class - RandomOverSampler 

In [42]:
from imblearn.over_sampling import RandomOverSampler

In [44]:
os =  RandomOverSampler(sampling_strategy=1)

In [45]:
X_train_res, y_train_res = os.fit_sample(X_train, y_train)

In [46]:
X_train_res.shape,y_train_res.shape

((452252, 42), (452252,))

In [47]:
print('Original dataset shape {}'.format(Counter(y_train)))
print('Resampled dataset shape {}'.format(Counter(y_train_res)))

Original dataset shape Counter({0: 226126, 1: 19882})
Resampled dataset shape Counter({0: 226126, 1: 226126})


## Logistic regression

In [48]:
logistic_regressor.fit(X_train_res,y_train_res)

LogisticRegression(C=2)

In [49]:
y_pred = logistic_regressor.predict(X_test)

In [50]:
n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Logistic Regression errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Logistic Regression errors: 23102
Accuracy Score :
0.624376046696909
Confusion matrix :
[[35825 20735]
 [ 2367  2576]]
Classification Report :
              precision    recall  f1-score   support

           0       0.94      0.63      0.76     56560
           1       0.11      0.52      0.18      4943

    accuracy                           0.62     61503
   macro avg       0.52      0.58      0.47     61503
weighted avg       0.87      0.62      0.71     61503

ROC AUC score is:  0.5772695843650144


## Random forest

In [51]:
random_forest.fit(X_train_res,y_train_res)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   56.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.0min finished


RandomForestClassifier(n_jobs=-1,
                       random_state=RandomState(MT19937) at 0x2211291CD08,
                       verbose=1)

In [52]:
y_pred = random_forest.predict(X_test)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.0s finished


In [53]:
n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Random Forest errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Random Forest errors: 4967
Accuracy Score :
0.9192397118839731
Confusion matrix :
[[56375   185]
 [ 4782   161]]
Classification Report :
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56560
           1       0.47      0.03      0.06      4943

    accuracy                           0.92     61503
   macro avg       0.69      0.51      0.51     61503
weighted avg       0.89      0.92      0.89     61503

ROC AUC score is:  0.5146502250836338


## XGBoost

In [54]:
X_train_res = pd.DataFrame(data=X_train_res, columns=selected_feat)

In [55]:
X_train_res.shape

(452252, 42)

In [56]:
xgb_classifier.fit(X_train_res,y_train_res)

XGBClassifier(max_depth=5)

In [57]:
y_pred = xgb_classifier.predict(X_test)

In [58]:
n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Extreme Gradient Boost errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Extreme Gradient Boost errors: 18046
Accuracy Score :
0.7065834186950231
Confusion matrix :
[[40144 16416]
 [ 1630  3313]]
Classification Report :
              precision    recall  f1-score   support

           0       0.96      0.71      0.82     56560
           1       0.17      0.67      0.27      4943

    accuracy                           0.71     61503
   macro avg       0.56      0.69      0.54     61503
weighted avg       0.90      0.71      0.77     61503

ROC AUC score is:  0.6900001459352316
